In [26]:
import random
import time
import torch
from torch import nn
import pickle
import glob
import argparse
import os

torch.cuda.empty_cache()

# os.environ["CURL_CA_BUNDLE"] = ""

# parser = argparse.ArgumentParser()
# parser.add_argument('-sp','--split', type=str, choices=['val_unseen', 'val_seen', 'tests_seen', 'tests_unseen', 'others'], required=True)
# parser.add_argument('-m','--model_saved_folder_name', type=str, required=True) # folder where the model is saved
# parser.add_argument('-o','--output_name', type=str, required=True) # save the output as a pickle file to this name
# parser.add_argument('--no_appended', action='store_true')

# args = parser.parse_args()
no_appended = True
if no_appended:
    granularity = 'noappended'
    name = 'high'
else:
    granularity = 'appended'
    name = 'high_low'

In [27]:

def accuracy(y_pred, y_batch):
    #y_pred has shape [batch, no_classes]
    maxed = torch.max(y_pred, 1)
    y_hat = maxed.indices
    num_accurate = torch.sum((y_hat == y_batch).long())
    train_accuracy = num_accurate/ y_hat.shape[0]
    return train_accuracy.item()

def accurate_both(y_pred1, y_batch1, y_pred2, y_batch2):
    #
    maxed1 = torch.max(y_pred1, 1)
    y_hat1 = maxed1.indices
    #
    maxed2 = torch.max(y_pred2, 1)
    y_hat2 = maxed2.indices
    #
    num_both_accurate = torch.sum((y_hat1 == y_batch1).long() * (y_hat2 == y_batch2).long())
    train_accuracy = num_both_accurate/ y_hat1.shape[0]
    return train_accuracy.item()


In [28]:
#Load data
import pickle
template_by_label = pickle.load(open('data/alfred_data/alfred_dicts/correct_template_by_label_ppdl.p', 'rb'))
new_labels_dict = pickle.load(open('data/alfred_data/alfred_dicts/correct_labels_dict_ppdl.p', 'rb'))
# val_set_unseen = pickle.load(open('data/alfred_data/'+ 'val_unseen_text_with_ppdl_low_appended.p', 'rb'))
val_set_unseen = pickle.load(open('../instruction2_params_val_unseen_noappended (actual).p', 'rb'))
val_set_seen = pickle.load(open('data/alfred_data/'+ 'val_seen_text_with_ppdl_low_appended.p', 'rb'))
val_set_seen

{'x': ['examine a bowl by the light of a desk lamp',
  'pick up a bowl and turn a lamp on',
  'pick up a brown bowl to look at under a lit lamp',
  'move a book from a desk to a sofa',
  'move the book to the couch ',
  'move the book from the table to the sofa',
  'turn on the desk lamp while holding a yellow disc',
  'grab the yellow stand on the table turn on the lamp',
  'examine the vase by lamplight',
  'to pick up a yellow object as well as turn on a lamp',
  'carry a cd while turning on a lamp',
  'take the cd from the table turn on the desk lamp',
  'put a cd on a desk',
  'examine the keys under the light of the table lamp',
  'inspect keys near a lit lamp',
  'take the keys on the dresser and turn on the lamp on the dresser',
  'look at a pillow by the light of a lamp',
  'examine the pillow in the light of the lamp on the table ',
  'pick up a pillow and turn a lamp on',
  'put a candle on the back of a toilet',
  'move a candle from a counter to the top of a toilet',
  'mo

In [29]:
#Set device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [30]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BertTokenizer, BertForSequenceClassification

import torch

model = T5ForConditionalGeneration.from_pretrained('t5-base')
model = model.to(device)
if no_appended:
    model.load_state_dict(torch.load('best_models/epoch_40_noappend.pt', map_location=device))
else:
    model.load_state_dict(torch.load('best_models/epoch_46_append.pt', map_location=device))
model.eval() # set model to evaluation mode

t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [31]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
bert_model = bert_model.to(device)
bert_model.load_state_dict(torch.load('best_models/sliced.pt', map_location=device))
bert_model.eval() # set model to evaluation mode

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [32]:
# get only the high level instructions string[0:[SEP]]

val_unseen_high_level_instructions = []
val_unseen_low_level_instructions = list(val_set_unseen.keys())

for key, value in val_set_unseen.items():
    val_unseen_high_level_instructions.append(key.split('[SEP]')[0])

if no_appended:
    validation_unseen = val_unseen_high_level_instructions
else:
    validation_unseen = val_unseen_low_level_instructions

In [33]:
labels2task_type = pickle.load(open('data/alfred_data/alfred_dicts/label2task_type.p', 'rb'))
labels2task_type
# flip the dictionary task_type : labels
task_type2labels = {labels2task_type[k]: k for k in labels2task_type}
task_type2labels

{'pick_and_place_simple': 2,
 'look_at_obj_in_light': 5,
 'pick_and_place_with_movable_recep': 1,
 'pick_two_obj_and_place': 3,
 'pick_clean_then_place_in_recep': 6,
 'pick_heat_then_place_in_recep': 4,
 'pick_cool_then_place_in_recep': 0}

In [34]:
validation_unseen

['pick up the cell phone and look at it by the light of the lamp',
 'turn on the bedroom lamp',
 'examine a phone with a lamp',
 'get an apple from the sink and heat it up in the microwave',
 'cook apple in microwave place in sink ',
 'put a cooked apple in the sink',
 'put a washed bowl away in a kitchen cabinet',
 'put a cleaned filled bowl of water in the cabinet  left of the microwave ',
 'put a clean wooden bowl of water in a cabinet',
 'put a spoon in a mug on a counter',
 'put a white mug with a spoon inside onto the counter',
 'put a cup with a spoon on the counter',
 'place the salt shaker in the cabinet',
 'put the salt in the cabinet above the counter ',
 'put the salt shaker in the cabinet to the left above the stove',
 'place two salt shakers in the drawer',
 'put two shakers in the second drawer',
 'put two shakers in the drawer to the left of the stove',
 'slice a tomato and cool a slice of it',
 'place a cold tomato slice on the counter',
 'slice and refrigerate a slice

In [35]:
obj2idx = pickle.load(open('data/alfred_data/alfred_dicts/obj2idx.p', 'rb'))
recep2idx = pickle.load(open('data/alfred_data/alfred_dicts/recep2idx.p', 'rb'))
labels2task_type = pickle.load(open('data/alfred_data/alfred_dicts/label2task_type.p', 'rb'))
objects = list(obj2idx.keys())
receptacles = list(recep2idx.keys())
task_types = list(labels2task_type.values())
# change None in the lists to 'None'
objects = ['None' if x is None else x for x in objects]
receptacles = ['None' if x is None else x for x in receptacles]
objects
# # remove 'Bottle' from objects
objects.remove('Bottle')
objects.remove('LaundryHamper')
objects.remove('Poster')
objects.remove('ShowerGlass')
objects.remove('ShowerHead')
# objects = {'AlarmClock': 0, 'Apple': 1, 'AppleSliced': 2, 'BaseballBat': 3, 'BasketBall': 4, 'Book': 5, 'Bowl': 6, 'Box': 7, 'Bread': 8, 'BreadSliced': 9, 'ButterKnife': 10, 'CD': 11, 'Candle': 12, 'CellPhone': 13, 'Cloth': 14, 'CreditCard': 15, 'Cup': 16, 'DeskLamp': 17, 'DishSponge': 18, 'Egg': 19, 'Faucet': 20, 'FloorLamp': 21, 'Fork': 22, 'Glassbottle': 23, 'HandTowel': 24, 'HousePlant': 25, 'Kettle': 26, 'KeyChain': 27, 'Knife': 28, 'Ladle': 29, 'Laptop': 30, 'LaundryHamperLid': 31, 'Lettuce': 32, 'LettuceSliced': 33, 'LightSwitch': 34, 'Mug': 35, 'Newspaper': 36,
#                     'Pan': 37, 'PaperTowel': 38, 'PaperTowelRoll': 39, 'Pen': 40, 'Pencil': 41, 'PepperShaker': 42, 'Pillow': 43, 'Plate': 44, 'Plunger': 45, 'Pot': 46, 'Potato': 47, 'PotatoSliced': 48, 'RemoteControl': 49, 'SaltShaker': 50, 'ScrubBrush': 51, 'ShowerDoor': 52, 'SoapBar': 53, 'SoapBottle': 54, 'Spatula': 55, 'Spoon': 56, 'SprayBottle': 57, 'Statue': 58, 'StoveKnob': 59, 'TeddyBear': 60, 'Television': 61, 'TennisRacket': 62, 'TissueBox': 63, 'ToiletPaper': 64, 'ToiletPaperHanger':65, 'ToiletPaperRoll': 66, 'Tomato': 67, 'TomatoSliced': 68, 'Towel': 69, 'Vase': 70, 'Watch': 71, 'WateringCan': 72, 'WineBottle': 73}
# objects = list(objects.keys())
# receptacles = {'ArmChair': 0, 'BathtubBasin': 1, 'Bed': 2, 'Cabinet': 3, 'Cart': 4, 'CoffeeMachine': 5, 'CoffeeTable': 6, 'CounterTop': 7, 'Desk': 8, 'DiningTable': 9, 'Drawer': 10,
#                                'Dresser': 11, 'Fridge': 12, 'GarbageCan': 13, 'Microwave': 14, 'Ottoman': 15, 'Safe': 16, 'Shelf': 17, 'SideTable': 18, 'SinkBasin': 19, 'Sofa': 20, 'StoveBurner': 21, 'TVStand': 22, 'Toilet': 23}
# receptacles = list(receptacles.keys())
print(receptacles)
print(objects)

['None', 'TowelHolder', 'Cup', 'CoffeeMachine', 'Sink', 'Ottoman', 'CounterTop', 'Desk', 'Sofa', 'Mug', 'CoffeeTable', 'ToiletPaperHanger', 'Pot', 'TVStand', 'Cart', 'Cabinet', 'Toilet', 'DiningTable', 'Safe', 'Bathtub', 'LaundryHamper', 'HandTowelHolder', 'Dresser', 'Pan', 'Plate', 'Box', 'BathtubBasin', 'StoveBurner', 'SideTable', 'Fridge', 'GarbageCan', 'DiningTable,TVStand', 'Toaster', 'Drawer', 'Bowl', 'Shelf', 'ArmChair', 'Sink,Sink', 'Bed', 'SinkBasin', 'TVStand,CoffeeTable', 'Microwave']
['AlarmClock', 'AluminumFoil', 'Apple', 'AppleSliced', 'ArmChair', 'BaseballBat', 'BasketBall', 'Bathtub', 'BathtubBasin', 'Bed', 'Blinds', 'Book', 'Boots', 'Bowl', 'Box', 'Bread', 'BreadSliced', 'ButterKnife', 'Cabinet', 'Candle', 'CD', 'CellPhone', 'Chair', 'Cloth', 'CoffeeMachine', 'CoffeeTable', 'CounterTop', 'CreditCard', 'Cup', 'Curtains', 'Desk', 'DeskLamp', 'Desktop', 'DiningTable', 'DishSponge', 'DogBed', 'Drawer', 'Dresser', 'Dumbbell', 'Egg', 'EggCracked', 'Faucet', 'Floor', 'FloorLa

matches the generated outputs to longest common subsequence of the two strings.

In [36]:
def longest_common_substring(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    longest = 0  # Length of the longest common substring
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                longest = max(longest, dp[i][j])
            else:
                dp[i][j] = 0
    
    return longest

def find_best_match_by_contiguous_substring(generated_target, desired_targets):
    best_match = None
    best_length = 0
    
    for target in desired_targets:
        substring_length = longest_common_substring(generated_target.lower(), target.lower())
        if substring_length > best_length:
            best_length = substring_length
            best_match = target
    
    return best_match

# Example usage
generated_target = "cool"
desired_targets = ['AlarmClock',
 'AluminumFoil',
 'Apple',
 'AppleSliced',
 'ArmChair',
 'BaseballBat',
 'BasketBall',
 'Bathtub',
 'BathtubBasin',
 'Bed',
 'Blinds',
 'Book',
 'Boots',
 'Bottle',
 'Bowl',
 'Box',
 'Bread',
 'BreadSliced',
 'ButterKnife',
 'Cabinet',
 'Candle',
 'CD',
 'CellPhone',
 'Chair',
 'Cloth',
 'WateringCan',
 'Window',
 'WineBottle',
 'Glassbottle']


best_match = find_best_match_by_contiguous_substring(generated_target, task_types)
print(f"The best match is: {best_match}")


The best match is: pick_cool_then_place_in_recep


In [37]:
val_unseen_high_preds_top1 = {}
val_unseen_high_preds_top2 = {}
val_unseen_high_preds_top3 = {}
val_unseen_high_preds_top4 = {}
val_unseen_high_preds_top5 = {}

for i in range(len(validation_unseen)):
    task_prefix = 'summarize the task parameters: '
    instruction = validation_unseen[i]
    # for t5 model 4 parameters (task_type, object_target, mrecep_target, parent_target)
    input_ids = t5_tokenizer(task_prefix + instruction, return_tensors='pt').input_ids.to(device)
    outputs = model.generate(input_ids, max_length=700, num_beams=5, num_return_sequences=5, early_stopping=False)

    for j, output in enumerate(outputs):
        predicted_parameters = t5_tokenizer.decode(output, skip_special_tokens=True)
        # for sliced bert model
        encoding_val_unseen = bert_tokenizer(validation_unseen[i], return_tensors='pt', padding=True, truncation=True, max_length=512)
        input_ids_val_unseen = encoding_val_unseen['input_ids'].to(device)
        attention_mask_val_unseen = encoding_val_unseen['attention_mask'].to(device)

        # debugging and process the predicted parameters
        # print(predicted_parameters)
        if j == 0:
            try:
                gen_task_type = predicted_parameters.split('task_type:')[1].split(',')[0]
                task_type = find_best_match_by_contiguous_substring(gen_task_type, task_types)
            except:
                gen_task_type = 'pick_and_place_simple'
            
        try:
            gen_object_target = predicted_parameters.split('object_target:')[1].split(',')[0]
            object_target = find_best_match_by_contiguous_substring(gen_object_target, objects)
        except:
            gen_object_target = None
        try:
            gen_mrecep_target = predicted_parameters.split('mrecep_target:')[1].split(',')[0]
            mrecep_target = find_best_match_by_contiguous_substring(gen_mrecep_target, receptacles)
        except:
            gen_mrecep_target = None
        try:
            gen_parent_target = predicted_parameters.split('parent_target:')[1].split(',')[0]
            parent_target = find_best_match_by_contiguous_substring(gen_parent_target, objects)
        except:
            gen_parent_target = None

        sliced = bert_model(input_ids_val_unseen, attention_mask=attention_mask_val_unseen).logits.argmax().item()
        
        if j == 0:
            val_unseen_high_preds_top1[val_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}
        elif j == 1:
            val_unseen_high_preds_top2[val_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}
        elif j == 2:
            val_unseen_high_preds_top3[val_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}
        elif j == 3:
            val_unseen_high_preds_top4[val_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}
        elif j == 4:
            val_unseen_high_preds_top5[val_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}


val_unseen_high_preds_top1


{'pick up the cell phone and look at it by the light of the lamp[SEP]walk forward to the dresser[SEP]pick up the cell phone from the dresser[SEP]turn left and go to the lamp[SEP]hold the cell phone and turn the lamp on': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'CellPhone',
  'parent_target': 'None',
  'sliced': 0},
 'turn on the bedroom lamp[SEP]walk to the dresser in front of you [SEP]pick up the cell phone above the left top drawer on the dresser [SEP]go to the lamp to your left in the corner of the room [SEP]turn on the lamp': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'Pen',
  'parent_target': 'None',
  'sliced': 0},
 'examine a phone with a lamp[SEP]walk forward to the dresser[SEP]pick up the phone that is on the dresser[SEP]take a couple step to the left to face lamp hold phone up[SEP]turn the lamp on': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'CellPhone

In [38]:
sample_ep = 477
key = list(val_unseen_high_preds_top1.keys())[sample_ep]

print('instruction:', key)
print('val_unseen_high_preds_top1:', val_unseen_high_preds_top1[key])
print('val_unseen_high_preds_top2:', val_unseen_high_preds_top2[key])
print('val_unseen_high_preds_top3:', val_unseen_high_preds_top3[key])
print('val_unseen_high_preds_top4:', val_unseen_high_preds_top4[key])
print('val_unseen_high_preds_top5:', val_unseen_high_preds_top5[key])

instruction: put a warmed up apple in the trash can[SEP]turn left and move to the end of the island above the trash can[SEP]pick up the green apple from the counter[SEP]turn left and move across the room to the microwave on the left[SEP]warm the apple in the microwave for a few seconds and then take it back out[SEP]turn left and head back to the end of the island where the trash can is located[SEP]throw the apple into the trash can
val_unseen_high_preds_top1: {'task_type': 'pick_heat_then_place_in_recep', 'mrecep_target': 'None', 'object_target': 'Apple', 'parent_target': 'GarbageBag', 'sliced': 0}
val_unseen_high_preds_top2: {'task_type': 'pick_heat_then_place_in_recep', 'mrecep_target': 'None', 'object_target': 'Apple', 'parent_target': 'GarbageBag', 'sliced': 0}
val_unseen_high_preds_top3: {'task_type': 'pick_heat_then_place_in_recep', 'mrecep_target': 'None', 'object_target': 'Apple', 'parent_target': 'GarbageBag', 'sliced': 0}
val_unseen_high_preds_top4: {'task_type': 'pick_heat_t

In [39]:
val_unseen_high_preds_top1_copy = val_unseen_high_preds_top1.copy()
val_unseen_high_preds_top2_copy = val_unseen_high_preds_top2.copy()
val_unseen_high_preds_top3_copy = val_unseen_high_preds_top3.copy()
val_unseen_high_preds_top4_copy = val_unseen_high_preds_top4.copy()
val_unseen_high_preds_top5_copy = val_unseen_high_preds_top5.copy()
val_unseen_high_preds = [val_unseen_high_preds_top1_copy, val_unseen_high_preds_top2_copy, val_unseen_high_preds_top3_copy, val_unseen_high_preds_top4_copy, val_unseen_high_preds_top5_copy]
# len(val_unseen_high_preds_copy)

In [40]:
val_unseen_high_preds_top2_copy

{'pick up the cell phone and look at it by the light of the lamp[SEP]walk forward to the dresser[SEP]pick up the cell phone from the dresser[SEP]turn left and go to the lamp[SEP]hold the cell phone and turn the lamp on': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'CellPhone',
  'parent_target': 'None',
  'sliced': 0},
 'turn on the bedroom lamp[SEP]walk to the dresser in front of you [SEP]pick up the cell phone above the left top drawer on the dresser [SEP]go to the lamp to your left in the corner of the room [SEP]turn on the lamp': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'Pillow',
  'parent_target': 'None',
  'sliced': 0},
 'examine a phone with a lamp[SEP]walk forward to the dresser[SEP]pick up the phone that is on the dresser[SEP]take a couple step to the left to face lamp hold phone up[SEP]turn the lamp on': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'CellPh

In [41]:
for i in val_unseen_high_preds:
    for j in i:
        if i[j]['task_type'] not in task_type2labels.keys():
            print('task_type not found:', i[j])
            # print(i[j])
            i[j]['task_type'] = 2 # set random task_type
        else:
            i[j]['task_type'] = task_type2labels[i[j]['task_type']]
        # print(i[j])
        # remove spaces from object_target
        # i[j]['object_target'] = i[j]['object_target'].replace(' ', '')
        # i[j]['mrecep_target'] = i[j]['mrecep_target'].replace(' ', '')
        # i[j]['parent_target'] = i[j]['parent_target'].replace(' ', '')
        # if 'None' change to None
        if i[j]['object_target'] == 'None':
            i[j]['object_target'] = None
        if i[j]['mrecep_target'] == 'None':
            i[j]['mrecep_target'] = None
        if i[j]['parent_target'] == 'None':
            i[j]['parent_target'] = None

In [42]:
val_unseen_high_preds[0]
val_unseen_high_preds[1]
val_unseen_high_preds[2]
val_unseen_high_preds[3]
val_unseen_high_preds[4]

{'pick up the cell phone and look at it by the light of the lamp[SEP]walk forward to the dresser[SEP]pick up the cell phone from the dresser[SEP]turn left and go to the lamp[SEP]hold the cell phone and turn the lamp on': {'task_type': 5,
  'mrecep_target': None,
  'object_target': 'CellPhone',
  'parent_target': None,
  'sliced': 0},
 'turn on the bedroom lamp[SEP]walk to the dresser in front of you [SEP]pick up the cell phone above the left top drawer on the dresser [SEP]go to the lamp to your left in the corner of the room [SEP]turn on the lamp': {'task_type': 5,
  'mrecep_target': None,
  'object_target': 'Box',
  'parent_target': None,
  'sliced': 0},
 'examine a phone with a lamp[SEP]walk forward to the dresser[SEP]pick up the phone that is on the dresser[SEP]take a couple step to the left to face lamp hold phone up[SEP]turn the lamp on': {'task_type': 5,
  'mrecep_target': None,
  'object_target': 'CellPhone',
  'parent_target': 'CoffeeTable',
  'sliced': 0},
 'get an apple from 

In [43]:
# save the output as a pickle file
pickle.dump(val_unseen_high_preds[0], open('val_unseen_T5_All_noappended_pred1.p', 'wb'))
pickle.dump(val_unseen_high_preds[1], open('val_unseen_T5_All_noappended_pred2.p', 'wb'))
pickle.dump(val_unseen_high_preds[2], open('val_unseen_T5_All_noappended_pred3.p', 'wb'))
pickle.dump(val_unseen_high_preds[3], open('val_unseen_T5_All_noappended_pred4.p', 'wb'))
pickle.dump(val_unseen_high_preds[4], open('val_unseen_T5_All_noappended_pred5.p', 'wb'))

In [44]:
len(val_unseen_high_preds)

5